In [1]:
%matplotlib inline
import time
import torch  as t
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable as V
from ipywidgets import interact, fixed
from IPython.display import clear_output
import gc
import random

def affine_transform(input,mat):
    coor=coordinate_one()
    xyzprime=t.tensordot(coor,t.t(mat),dims=1)
    return trilinear(input,xyzprime)

def coordinate(shape):
    coor=np.asarray(np.meshgrid(np.arange(X), np.arange(Y), np.arange(Z)))
    coor=np.rollaxis(coor,0,2)
    coor=np.rollaxis(coor,0,3)
    coor=np.rollaxis(coor,0,4)
    return t.tensor(coor,device='cuda')

def coordinate_one():
    coor=np.asarray(np.meshgrid(np.arange(X), np.arange(Y), np.arange(Z)))
    coor=np.rollaxis(coor,0,2)
    coor=np.rollaxis(coor,0,3)
    coor=np.rollaxis(coor,0,4)
    onesMatrix = np.ones((X,Y,Z,1))
    coorMatrix = np.concatenate((coor, onesMatrix), axis=3)
    return t.tensor(coorMatrix,device='cuda')
    
def trilinear(img,xyzprime):
    xp=xyzprime[:,:,:,0]
    yp=xyzprime[:,:,:,1]
    zp=xyzprime[:,:,:,2]
    LUX=t.floor(xp).int()
    LUY=t.floor(yp).int()
    LUZ=t.floor(zp).int()
    LUX=t.where(LUX>X-2,t.tensor(X-2,device='cuda'),LUX.long())
    LUX=t.where(LUX<1,t.tensor(1,device='cuda'),LUX)
    LUY=t.where(LUY>Y-2,t.tensor(Y-2,device='cuda'),LUY.long())
    LUY=t.where(LUY<1,t.tensor(1,device='cuda'),LUY)
    LUZ=t.where(LUZ>Z-2,t.tensor(Z-2,device='cuda'),LUZ.long())
    LUZ=t.where(LUZ<1,t.tensor(1,device='cuda'),LUZ)
    xd,yd,zd=yp-LUY.double(),xp-LUX.double(),zp-LUZ.double()
    k000,k001,k010,k011=img[LUX,LUY,LUZ],img[LUX,LUY,LUZ+1],img[LUX,LUY+1,LUZ],img[LUX,LUY+1,LUZ+1]
    k100,k101,k110,k111=img[LUX+1,LUY,LUZ],img[LUX+1,LUY,LUZ+1],img[LUX+1,LUY+1,LUZ],img[LUX+1,LUY+1,LUZ+1]
    i1=k000*(1-zd)+k001*zd
    i2=k010*(1-zd)+k011*zd
    j1=k100*(1-zd)+k101*zd
    j2=k110*(1-zd)+k111*zd
    w1=i1*(1-yd)+i2*yd
    w2=j1*(1-yd)+j2*yd
    res=w1*(1-xd)+w2*xd
#     del w1,w2,i1,i2,j1,j2,k000,k001,k010,k011,k100,k101,k110,k111,xd,yd,zd,LUX,LUY,LUZ,xp,yp,zp
    return res
    
def randxyz(shape,rate):
    np.random.random(shape)
    rst=t.empty(shape).uniform_(0, 1)
    return t.where(rst>t.tensor(rate),t.tensor(0),t.tensor(1))
    
def np_randxyz(shape, rate):
    rst=np.random.random(shape)
    rst=np.where(rst>rate,0,1)
    return t.tensor(rst,device='cuda')
    
def gradient_check(img, template, matrix, gradient):
    epsilon=1e-13
    difference=t.zeros(matrix.shape)
    for i in range(len(matrix)-1):
        for j in range(len(matrix[0])):
            theltaplus=t.tensor(matrix,device='cuda')
            theltaminus=t.tensor(matrix,device='cuda')
            theltaplus[i][j]+=epsilon
            theltaminus[i][j]-=epsilon
            J_plus=t.sum((template-affine_transform(img,theltaplus))**2)/(X*Y*Z)
            J_plus=J_plus.cpu().detach().numpy()
            J_minus=t.sum((template-affine_transform(img,theltaminus))**2)/(X*Y*Z)
            J_minus=J_minus.cpu().detach().numpy()
            gradapprox=(J_plus-J_minus)/(2*epsilon)
            numerator = np.linalg.norm(gradient[i][j] - gradapprox) 
            denominator = np.linalg.norm(gradient[i][j]) + np.linalg.norm(gradapprox) 
            difference[i][j]=numerator/denominator
    print(difference)
    
def display_images(image_z, moving, transformed, template):
    plt.subplots(1,3,figsize=(10,8))
    
    plt.subplot(1,3,1)
    plt.imshow(moving[image_z,:,:],cmap=plt.cm.Greys_r);
    plt.title('moving image')
    plt.axis('off')
    
    plt.subplot(1,3,2)
    plt.imshow(transformed[image_z,:,:],cmap=plt.cm.Greys_r);
    plt.title('transformed image')
    plt.axis('off')
    
    plt.subplot(1,3,3)
    plt.imshow(template[image_z,:,:],cmap=plt.cm.Greys_r);
    plt.title('template image')
    plt.axis('off')
    
    plt.show()
    
def scale(ori, f):
    mov=np.zeros((int(ori.shape[0]/f),int(ori.shape[1]/f),int(ori.shape[2]/f)))
    for i in range(len(ori)):
        if i%f==0:
            for j in range(len(ori[0])):
                if j%f==0:
            for k in range(len(ori[0][0])):
                if k%f==0:
                    mov[int(i/2)][int(j/2)][int(k/2)]=ori[i][j][k]
    return mov
    
def interact_display(m,transformed,template):
    interact(display_images, image_z=(0,m.shape[0]-1,10),
         moving = fixed(m.cpu().detach().numpy()), 
         transformed=fixed(transformed.cpu().detach().numpy()), 
         template=fixed(template.cpu().detach().numpy()))


In [2]:
all=np.load("MRI_3D/OAS1_0001_MR1_mpr_n4_anon_111_t88_masked_gfc_rep1.npz")
mov=all["moving"]
mov=V(t.tensor(mov).double(),True)#.cuda()
X,Y,Z=mov.shape
# template=all["fixed"]
# template=V(t.tensor(template).double(),True).cuda()
transformMatrix=V(t.tensor(np.array([[1.1,0,0,0],
                                     [0,1.1,0,0],
                                     [0,0,1.1,0],
                                     [0,0,0,1]])).double(), True).cuda()
template=affine_transform(mov,transformMatrix)


RuntimeError: CUDA out of memory. Tried to allocate 52.50 MiB (GPU 0; 2.00 GiB total capacity; 1.38 GiB already allocated; 34.77 MiB free; 1023.50 KiB cached)

In [ ]:
mat=V(t.tensor(np.array([[1,0,0,0],
                         [0,1,0,0],
                         [0,0,1,0],
                         [0,0,0,1]])).double(), True).cuda()
step=5e-4
iter=100
rate=1e-4
losses=[]
start=time.clock()
for i in range(iter):    
    tmplt=affine_transform(mov,mat)
    err=(template-tmplt)
    loss=t.sum(err[randxyz(mov.shape,rate)==1]**2)/(X*Y*Z)   
    loss.backward(retain_graph=True)
    losses.append(np.sum((tmplt.detach().numpy()-template.detach().numpy())**2)/(Z*X*Y))

    with t.no_grad():
#         mat[:3,:3]-=step*mat.grad[:3,:3]
#         mat[:3,3]-=step*mat.grad[:3,3]
        mat-=step*mat.grad
#         print(mat.grad)
        mat.grad.zero_()
#     gc.collect()
plt.plot(np.arange(iter), losses)
print(time.clock()-start)